# Contributing your data to Refinitiv with Elektron WebSocket API



## RCC Overview

The Refinitiv Contribution Channel (RCC - formerly known as TRCC)  is a new service for on-boarding content to the Refinitiv. Depending on the client's needs, access to the service will be fulfilled by one or more of the following products: Contributions Channel for TREP, Contributions Channel for Elektron API, Contributions Channel for Spreadsheet. RCC aims for replacing the legacy Market Link IP (MLIP) system.

Data Contribution is a means to send your pricing data directly to Refinitiv , from where it can fan out globally. Leading market participants, such as banks and brokers, use Refinitiv to publish and distribute their information to the community of financial professions, in order to advertise their prices to clients and prospects.

## Application Overview

This example shows how to writing an application to contribute your data to RCC using [Elektron WebSocket API](https://developers.refinitiv.com/elektron/websocket-api) through Thomson Reuters Enterprise Platform (TREP). The example just connects to TREP via a WebSocket connection, then sends an off-stream post to contribute item to RCC server via that TREP.

If you are not familiar with Elektron WebSocket API Posting concept, please visit [Contributing Data to TREP using the Websocket API](https://developers.refinitiv.com/article/contributing-data-trep-using-websocket-api) article which will give you a full explanation of the WebSocket API Posting mechanisms and process.

## Contribution Setups

The Elektron WebSocket API cannot connect to RCC server directly (only Elektron SDK C++ and Java APIs support that feature). It requires TREP version 3.2.1 (and above) to take care of the JSON-OMM conversion, RCC connection and login process for the WebSocket application.

The TREP infrastructure connects to RCC through the delivery direct network via Tunnel Stream Aggregator (TSA) adapter, which is a private network (TLS encrypted) between a client site and Refinitiv. The TSA adapter is already packaged with the ADH version 3.2, and needs to be configured. You can find more detail regarding the TREP-RCC configurations in [Contributing your data to Refinitiv article](https://developers.refinitiv.com/article/contributing-your-data-thomson-reuters) page (*Contribution through TREP* section). 


## Prerequisite

This example requires the following dependencies softwares and libraries.
1. TREP server (both ADS and ADH) 3.2.x with WebSocket connection.
2. RCC username, password and host list credentials. Please reach out to your Refinitiv sales associate to acquire RCC access credentials.

*Note:* The RCC access credentials are required in the connection between ADH server and RCC server only, not in the application level.

## How to run this console example

Please be informed that your TREP server (ADS and ADH) should be applied the RCC configurations and RCC contribution service should be "Up" before running an example. 

In [1]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed
# #Install requests package in a current Jupyter kernal

# import sys
# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [2]:
# import required libraries

import sys
import time
import getopt
import socket
import json
import websocket
import threading
import os
from threading import Thread, Event

In [9]:
# TREP connection variables

hostname = '127.0.0.1'
port = '15000'
user = 'root'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [10]:
# RCC contribution variables

post_item_name = 'CONTRIBUTION_RIC'
service_name = 'TRCC'

post_id = 1
bid_value = 34.25
ask_value = 35.48
primact_1_value = 116.50

In [11]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False

## Contribution Process

The application requires the following steps to contribute data to RCC via TREP:
1. The application initiates WebSocket connection with ADS server.
2. Application sends a OMM Login request message to ADS in JSON format.
3. Once the application receives a Login Refresh message from ADS, the application can contribute data to RCC via Off-Stream Post message.
4. The Off-Stream Post message must have the following conditions:
    * The ```Ack``` attribute must be true
    * The message must contain the ```PostID``` attribute and value
    * The ```Key``` information which includes ```Name``` and ```Service``` attributes that refer to contribution RIC name and RCC contribution service name
    * The ```Message``` payload must be an Update message type
    * The ```Message``` payload must contain the same ```Key``` information as the Post message

In [12]:
# JSON-OMM Login and Process messages

def process_message(ws, message_json):  # Process all incoming messages.
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if 'Domain' in message_json:
            message_domain = message_json['Domain']
            if message_domain == "Login":
                process_login_response(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))
        
# Process incoming Login Refresh Response message.
def process_login_response(ws, message_json):
    """ Send Off-Stream Post """
    print("Sending Off-Stream Post to TREP Server")
    send_market_price_post(ws)    
    
# Create JSON Login request message and sends it to ADS server.
def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': login_id,
        'Domain': 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

## Off-Stream Post

In an off-stream post, the client application can send a post for an item via a Login stream, regardless of whether a data stream first exists. The route of the post is determined by the TREP (ADS and ADH) configuration.

In [13]:
# Create JSON Off-Stream Post message and sends it to ADS server.

def send_market_price_post(ws):
    """ Send a post message contains a market-price content to RCC """

    """ Contribution fields """
    contribution_fields = {
        "BID": bid_value,
        "ASK": ask_value,
        "PRIMACT_1": primact_1_value
    }

    """ OMM Post msg Key """
    mp_post_key = {
        "Name": post_item_name,
        "Service": service_name
    }

    """ OMM Post Payload """
    contribution_payload_json = {
        "ID": 0,
        "Type": "Update",
        "Domain": "MarketPrice",
        "Fields": contribution_fields,
        "Key": {}
    }

    """ OMM Off-Stream Post message """
    mp_post_json_offstream = {
        "Domain": "MarketPrice",
        "Ack": True,
        "PostID": post_id,
        "PostUserInfo": {
            "Address": position,
            "UserID": int(app_id)
        },
        "Key": {},
        "Message": {},
        "Type": "Post",
        "ID": login_id #Off-Stream post, sending the Post message on the Loing stream.
    }

    contribution_payload_json["Key"] = mp_post_key
    mp_post_json_offstream["Key"] = mp_post_key
    mp_post_json_offstream["Message"] = contribution_payload_json

    ws.send(json.dumps(mp_post_json_offstream))
    print("SENT:")
    print(json.dumps(mp_post_json_offstream, sort_keys=True, indent=2, separators=(',', ':')))


In [14]:
# WebSocket connection process

def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(10)
    web_socket_app.close()


Connecting to WebSocket ws://172.20.33.30:15000/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "Position":"10.42.68.162"
    },
    "Name":"root"
  }
}
RECEIVED: 
[
  {
    "Domain":"Login",
    "Elements":{
      "MaxMsgSize":61430,
      "PingTimeout":30
    },
    "ID":1,
    "Key":{
      "Elements":{
        "AllowSuspectData":1,
        "ApplicationId":"256",
        "ApplicationName":"ADS",
        "Position":"10.42.68.162",
        "ProvidePermissionExpressions":1,
        "ProvidePermissionProfile":0,
        "SingleOpen":1,
        "SupportBatchRequests":7,
        "SupportEnhancedSymbolList":1,
        "SupportOMMPost":1,
        "SupportOptimizedPauseResume":1,
        "SupportPauseResume":1,
        "SupportStandby":1,
        "SupportViewRequests":1
      },
      "Name":"root"
    },
    "State":{
      "Data":"Ok",
      "Stream":"Open",
      "Text":"Login accepted by h

## References
* [Refinitiv Elektron SDK Family page](https://developers.refinitiv.com/elektron) on the [Refinitiv Developer Community](https://developers.refinitiv.com/) web site.
* [Refinitiv Elektron WebSocket API page](https://developers.refinitiv.com/websocket-api).
* [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
* [Contributing Data to TREP using the Websocket API article](https://developers.refinitiv.com/article/contributing-data-trep-using-websocket-api).
* [Contributing your data to Refinitiv article](https://developers.refinitiv.com/article/contributing-your-data-thomson-reuters).
* [Refinitiv Elektron: RIC Search](https://developers.refinitiv.com/elektron/websocket-api/dev-tools?type=ric).
* [Refinitiv Data Model Discovery page](https://refinitiv.fixspec.com/specserver/specs/reuters): Explore TR data models, content definitions and data update behaviors.
* [EMA Java Tutorial - Posting data to Contribution Channel](https://developers.refinitiv.com/elektron/elektron-sdk-java/learning?content=41885&type=learning_material_item).
* [EMA C++ Tutorial - Posting data to Contribution Channel](https://developers.refinitiv.com/elektron/elektron-sdk-cc/learning?content=26452&type=learning_material_item).

For any question related to this example or Elektron WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/152/websocket-api.html).